In [1]:
import pandas as pd
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials

In [2]:
luisappid = os.environ['LUIS_APP_ID']
luisapikey = os.environ['LUIS_API_KEY']
luisapihostname = os.environ['LUIS_API_HOST_NAME']

In [3]:
df_data = pd.read_json('../data/data_train.json', orient='records')

In [4]:
df_data.head()

,text,entityLabels,intentName
0,"Please, please book this terrific trip","[{'startCharIndex': 15, 'endCharIndex': 18, 'e...",book
1,That sounds great! I'll book it!,"[{'startCharIndex': 24, 'endCharIndex': 27, 'e...",book
2,Fine. What about...if I went to Milan instead,"[{'startCharIndex': 32, 'endCharIndex': 36, 'e...",book
3,Is there anything sooner for flights to Lima?,"[{'startCharIndex': 18, 'endCharIndex': 23, 'e...",book
4,Just one - just me,"[{'startCharIndex': 5, 'endCharIndex': 7, 'ent...",book


In [5]:
for it, sample in df_data.iterrows():
    if it < 40:
        #print(sample.to_dict())
        entity = sample.to_dict()['entityLabels']

In [6]:
def extract_in_sentence(entityLabels, text):
    dict_list = {}
    for entity in entityLabels:
        word, name = extract_entity(entity['startCharIndex'], entity['endCharIndex'], entity['entityName'], text)
        dict_list[word] = name
    dict_list['text'] = text
    return dict_list

In [7]:
def extract_entity(start, end, name, text):
    word = text[start:end+1]
    return name, word

In [8]:
df_list = []
for it, sample in df_data.iterrows():
    if it < 40:
        df_list.append({})
        df_list[it].update(extract_in_sentence(sample.to_dict()['entityLabels'], sample.to_dict()['text']))

In [9]:
df_list

[{'intent': 'book', 'text': 'Please, please book this terrific trip'},
 {'intent': 'book', 'text': "That sounds great! I'll book it!"},
 {'dst_city': 'Milan',
  'text': 'Fine. What about...if I went to Milan instead'},
 {'str_date': 'sooner',
  'text': 'Is there anything sooner for flights to Lima?'},
 {'n_adults': 'one', 'text': 'Just one - just me'},
 {'dst_city': 'Dublin', 'text': 'to Dublin .'},
 {'or_city': 'Gotham City,',
  'dst_city': 'Hogsmead',
  'n_children': '3',
  'n_adults': '9',
  'text': "Hi I need a break from Gotham City, the crime rate is just too high. I'd like to go to Hogsmead with 3 kids and 9 adults."},
 {'budget': '6000',
  'category': '5',
  'text': 'I’m willing to spend up to 6000 if it means I get a 5 star'},
 {'intent': 'book',
  'max_duration': '5 day',
  'or_city': 'Santa Cruz',
  'str_date': 'September 2nd.',
  'text': "Hi! I want to book a 5 day trip leaving from Santa Cruz on September 2nd. I don't have a destination in mind yet, what would you recommen

In [10]:
app_id = luisappid
predictionKey = luisapikey
predictionEndpoint = "https://" + luisapihostname


In [11]:
#predictionRequest = {
#    "query": df_list[39]['text']}
runtimeCredentials = CognitiveServicesCredentials(predictionKey)
clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)
#predictionResponse = clientRuntime.prediction.resolve(app_id, query=predictionRequest)

In [12]:
#df_list[39]['text']

In [13]:
    #runtimeCredentials = CognitiveServicesCredentials(predictionKey)
    #clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)

In [14]:
list_response = []

In [15]:
for it in range(8):
    list_response.append({})
    predictionRequest = {
    "query": df_list[it+30]['text']}
    predictionResponse = clientRuntime.prediction.resolve(app_id, query=predictionRequest)
    for x in range(len(predictionResponse.entities)):
        list_response[it][predictionResponse.entities[x].type] = \
        predictionResponse.entities[x].entity

In [16]:
list_response

[{'intent': 'book'},
 {'max_duration': '4 day',
  'builtin.datetimeV2.duration': '4 day',
  'builtin.number': '4'},
 {'or_city': 'tijuana ,', 'builtin.geographyV2.city': 'tijuana'},
 {'n_adults': 'one', 'builtin.number': 'one'},
 {'dst_city': 'santos',
  'n_adults': 'with a friend .',
  'builtin.geographyV2.city': 'santos'},
 {'dst_city': 'queenstown',
  'or_city': 'fort lauderdale',
  'builtin.geographyV2.city': 'queenstown'},
 {'max_duration': '8 days',
  'or_city': 'dallas .',
  'str_date': 'september 6',
  'builtin.datetimeV2.duration': '8 days',
  'builtin.datetimeV2.daterange': 'starting on september 6',
  'builtin.number': '6'},
 {'budget': '2800',
  'n_adults': 'just me .',
  'builtin.geographyV2.continent': 'south america',
  'builtin.number': '2800'}]

In [17]:
df_list[30:38]

[{'intent': 'book', 'text': 'Would I still be able to book that package?'},
 {'max_duration': '4 day', 'text': 'I want to go on a 4 day trip'},
 {'or_city': 'Tijuana,', 'text': 'I am in Tijuana, send me anywhere nice!'},
 {'n_adults': 'one', 'text': 'Great. Book for one adult.'},
 {'n_adults': 'with a friend.',
  'dst_city': 'Santos',
  'text': 'I’m traveling with a friend. Santos then'},
 {'or_city': 'Fort Lauderdale',
  'text': 'She lives in Fort Lauderdale and needs to go to Queenstown'},
 {'max_duration': '8 days',
  'str_date': 'september 6',
  'or_city': 'dallas.',
  'text': 'I have 8 days for vacation starting on september 6 from dallas. What can you offer me?'},
 {'n_adults': 'just me.',
  'budget': '2800',
  'text': "Yeah just me. I've just spent all this money backpacking around South America, so I've only got 2800 left. Can you find something with that?"}]

In [18]:
import re

In [52]:
table = {}
#table['tests'] = 0
for it in range(8):
    for key in df_list[30 + it].keys():
        real = ''
        predict = ''
        if not key == 'text':
            if not key in table.keys():
                table[key] = [0, 0, 0, 0, 0]  # TP FP TN FN SUM
            if key in list_response[it].keys():
                #table['tests'] += 1
                table[key][4] += 1  # Compte le nombre total d’apparition
                #print(key, ''.join(re.findall('\w*', df_list[it + 30][key])).lower())
                #print(key, ''.join(re.findall('\w*', list_response[it][key])).lower())
                real = ''.join(re.findall('\w*', df_list[it + 30][key])).lower()
                predict = ''.join(re.findall('\w*', list_response[it][key])).lower()
                if real == predict:
                    table[key][0] += 1  # Compte le nombre de bonne prédiction
                else:
                    table[key][1] += 1
            else:
                table[key][1] += 1
                table[key][4] += 1
    for key in list_response[it].keys():
        real = ''
        predict = ''
        if not key == 'text':
            if not key in table.keys():
                table[key] = [0, 0, 0, 0, 0]  # TP FP TN FN SUM 
            if key in df_list[30 + it].keys():
                #table['tests'] += 1
                table[key][4] += 1  # Compte le nombre total d’apparition
                #print(key, ''.join(re.findall('\w*', df_list[it + 30][key])).lower())
                #print(key, ''.join(re.findall('\w*', list_response[it][key])).lower())
                real = ''.join(re.findall('\w*', df_list[it + 30][key])).lower()
                predict = ''.join(re.findall('\w*', list_response[it][key])).lower()
                if real == predict:
                    table[key][2] += 1  # Compte le nombre de bonne prédiction
                else:
                    table[key][3] += 1
            else:
                table[key][3] += 1
                table[key][4] += 1


In [53]:
import matplotlib.pyplot as plt

In [54]:
table

{'intent': [1, 0, 1, 0, 2],
 'max_duration': [2, 0, 2, 0, 4],
 'builtin.datetimeV2.duration': [0, 0, 0, 2, 2],
 'builtin.number': [0, 0, 0, 4, 4],
 'or_city': [3, 0, 3, 0, 6],
 'builtin.geographyV2.city': [0, 0, 0, 3, 3],
 'n_adults': [3, 0, 3, 0, 6],
 'dst_city': [1, 0, 1, 1, 3],
 'str_date': [1, 0, 1, 0, 2],
 'builtin.datetimeV2.daterange': [0, 0, 0, 1, 1],
 'budget': [1, 0, 1, 0, 2],
 'builtin.geographyV2.continent': [0, 0, 0, 1, 1]}

In [55]:
df = pd.DataFrame.from_dict(table, orient='index', columns=['TP', 'FP', 'TN', 'FN', 'Total'])

In [56]:
df

,TP,FP,TN,FN,Total
intent,1,0,1,0,2
max_duration,2,0,2,0,4
builtin.datetimeV2.duration,0,0,0,2,2
builtin.number,0,0,0,4,4
or_city,3,0,3,0,6
builtin.geographyV2.city,0,0,0,3,3
n_adults,3,0,3,0,6
dst_city,1,0,1,1,3
str_date,1,0,1,0,2
builtin.datetimeV2.daterange,0,0,0,1,1


Ce résultat montre que sur un jeu de données de tests, LUIS ne fait aucune faute. Nous pourrons faire les tests en interface graphique.